In [0]:
import keras, os, pickle, re, sklearn, string, tensorflow
import numpy as np
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras.layers import Embedding
from keras.optimizers import Adadelta
from sklearn.model_selection import train_test_split


print('Keras version: \t\t%s' % keras.__version__)
print('Scikit version: \t%s' % sklearn.__version__)
print('TensorFlow version: \t%s' % tensorflow.__version__)


Using TensorFlow backend.


Keras version: 		2.2.4
Scikit version: 	0.21.2
TensorFlow version: 	1.13.1


In [0]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
nltk.download('punkt')   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence,remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')# Download text data sets, including stop words
from nltk.corpus import stopwords # Import the stop word list
import re
from bs4 import BeautifulSoup

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
def review_to_wordlist( review, remove_stopwords=False ):
# Function to convert a document to a sequence of words,
# optionally removing stop words. Returns a list of words.
#
# 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
#
# 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
#
# 3. Convert words to lower case and split them
    words = review_text.lower().split()
#
# 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
#
# 5. Return a list of words
    return(words)


In [0]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
nltk.download('punkt')
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
# Function to split a review into parsed sentences. Returns a
# list of sentences, where each sentence is a list of words
#
# 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
#
# 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
# If a sentence is empty, skip it
        if len(raw_sentence) > 0:
# Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, remove_stopwords ))
#
# Return the list of sentences (each sentence is a list of words,
# so this returns a list of lists
    return sentences


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
def read_files(path):
    documents = list()
    print("11111")
# Read in all files in directory
    if os.path.exists(path):
        print("22")
        for filename in os.listdir(path):
            with open('%s/%s' % (path, filename),encoding='utf-8') as f:

                doc = f.read()
                # doc = clean_text(doc)
                documents.append(doc)
# Read in all lines in a txt file
    if os.path.isfile(path):
        with open(path, encoding='iso-8859-1') as f:
            doc = f.readlines()
            for line in doc:
                documents.append(line)
    return documents


In [7]:
from google.colab import drive
import glob
import os
'''
drive.mount('/content/gdrive/')
!ls "/content/gdrive/My Drive"
download_path = os.path.expanduser("/content/gdrive/My Drive")
try:
  os.makedirs(download_path)
except FileExistsError:
  pass
'''
drive.mount('/content/gdrive/')


def read_files1(path):
    
    #drive.mount('/content/gdrive/')

    !mkdir drive
    !google-drive-ocamlfuse drive
    !ls "gdrive/My Drive/aclImdb"

    print(path)
    !ls path
    #path = glob.glob('/gdrive/aclImdb/train/pos/')
    print(path)
    documents = list()
    print("11111")
    i = 0
    for filename in os.listdir(path):
      with open('%s/%s' % (path, filename),encoding='utf-8') as f:
        print("2222")
        i = i+1
        print(i)
        doc = f.read()
        documents.append(doc)
        if i == 4:
          break
        
    if os.path.isfile(path):
        with open(path, encoding='iso-8859-1') as f:
            doc = f.readlines()
            for line in doc:
                documents.append(line)
    
    return documents
  

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def read_file(path):

  # 1. Authenticate and create the PyDrive client.
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

  # choose a local (colab) directory to store the data.
  local_download_path = os.path.expanduser('/gdrive/aclImdb/train/pos/')
  try:
    os.makedirs(local_download_path)
  except: pass

  # 2. Auto-iterate using the query syntax
  #    https://developers.google.com/drive/v2/web/search-parameters
  file_list = drive.ListFile(
      {'q': path }).GetList()
  i=0
  for f in file_list:
      
      #3. Create & download by id.
      #print('title: %s, id: %s' % (f['title'], f['id']))
      fname = os.path.join(local_download_path, f['title'])
      #print('downloading to {}'.format(fname))
      f_ = drive.CreateFile({'id': f['id']})
      f_.GetContentFile(fname)
      i =i+1
     
      documents = []
      with open(fname, 'r') as f:
        #doc = f.read()
        #documents.append(doc)
    # Read in all lines in a txt file
    #with open(path, encoding='iso-8859-1') as f:
    
        doc = f.readlines()
        for line in doc:
          documents.append(line)
      if( i == 4):
          break

      
  return file_list

In [0]:
a = read_file("'1-L4mvE7yg0UzcXwUHe2p7kKOBFgU9SsI' in parents")


In [0]:
import os
os.path.isfile("/content/gdrive/My Drive/CV")

False

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


E: Package 'python-software-properties' has no installation candidate
··········


In [0]:
!mkdir gdrive
!google-drive-ocamlfuse drive
!ls "/content/gdrive/My Drive/aclImdb/train/pos"

mkdir: cannot create directory ‘gdrive’: File exists
/bin/bash: google-drive-ocamlfuse: command not found
ls: cannot access '/content/gdrive/My Drive/aclImdb/train/pos': No such file or directory


In [0]:
f = open("/content/gdrive/My Drive/aclImdb/train/pos/0_9.txt")
print(f.read())


Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!


In [11]:
pos_train = read_files1('/content/gdrive/My Drive/aclImdb/train/pos')
neg_train = read_files1('/content/gdrive/My Drive/aclImdb/train/neg')
pos_test = read_files1('/content/gdrive/My Drive/aclImdb/test/pos')
neg_test = read_files1('/content/gdrive/My Drive/aclImdb/test/neg')
train1 = pos_train + neg_train
test1 = pos_test + neg_test

#docs = negative_docs + positive_docs
l_train = [1 for _ in range(len(pos_train))] + [0 for _ in range(len(neg_train))]
l_test = [1 for _ in range(len(pos_train))] + [0 for _ in range(len(neg_test))]
train = np.column_stack((train1,l_train))
test = np.column_stack((test1,l_test))

mkdir: cannot create directory ‘drive’: File exists
/bin/bash: google-drive-ocamlfuse: command not found
imdbEr.txt  imdb.vocab	README	test  train
/content/gdrive/My Drive/aclImdb/train/pos
ls: cannot access 'path': No such file or directory
/content/gdrive/My Drive/aclImdb/train/pos
11111
2222
1
2222
2
2222
3
2222
4
mkdir: cannot create directory ‘drive’: File exists
/bin/bash: google-drive-ocamlfuse: command not found
imdbEr.txt  imdb.vocab	README	test  train
/content/gdrive/My Drive/aclImdb/train/neg
ls: cannot access 'path': No such file or directory
/content/gdrive/My Drive/aclImdb/train/neg
11111
2222
1
2222
2
2222
3
2222
4
mkdir: cannot create directory ‘drive’: File exists
/bin/bash: google-drive-ocamlfuse: command not found
imdbEr.txt  imdb.vocab	README	test  train
/content/gdrive/My Drive/aclImdb/test/pos
ls: cannot access 'path': No such file or directory
/content/gdrive/My Drive/aclImdb/test/pos
11111
2222
1
2222
2
2222
3
2222
4
mkdir: cannot create directory ‘drive’: File 

OSError: ignored

In [10]:
# probably to solve ERROR[5]
!pip install fastai
!pip install opencv-python
!apt update && apt install -y libsm6 libxext6
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
!pip3 install torchvision


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [56.4 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 

In [0]:
print(documents)

NameError: ignored

In [0]:
sentences = [] # Initialize an empty list of sentences
num_reviews = 25000
print ("Parsing sentences from training set")
for review in range( 0, num_reviews ):
    sentences += review_to_sentences(train[review][0], tokenizer)
'''
print ("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)
'''

Parsing sentences from training set


IndexError: ignored

In [0]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',    level=logging.INFO)

def myhashfxn(obj):
    return hash(obj) % (2 ** 32)

# Set values for various parameters
num_features = 400    # Word vector dimensionality                      
min_word_count = 60   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words


#**********************************************************



# Initialize and train the model (this will take some time)
from gensim.models import word2vec
#model = word2vec.Word2Vec(hashfxn=myhashfxn)

print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers,             size=num_features, min_count = min_word_count,             window = context, sample = downsampling,hashfxn=myhashfxn)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)


In [0]:
model.doesnt_match("man woman child kitchen".split())


In [0]:

model.most_similar("man")



In [0]:
#Vector Averaging
import numpy as np # Make sure that numpy is imported
def makeFeatureVec(words, model, num_features):
# Function to average all of the word vectors in a given
# paragraph
#
# Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
#
    nwords = 0.
#
# Index2word is a list that contains the names of the words in
# the model's vocabulary. Convert it to a set, for speed
    index2word_set = set(model.wv.index2word)
#
# Loop over each word in the review and, if it is in the model's
# vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
#
# Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec



In [0]:
def getAvgFeatureVecs(reviews, model, num_features):
# Given a set of reviews (each one a list of words), calculate
# the average feature vector for each one and return a 2D numpy array
#
# Initialize a counter
    counter = 0
#
# Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
# Loop through the reviews
    for review in reviews:
        if counter%1000 == 0:
            print ("Review %d of %d" % (counter, len(reviews)))
        reviewFeatureVecs[counter] = makeFeatureVec(review, model,num_features)
#
# Increment the counter
        counter = counter + 1
    return reviewFeatureVecs




In [0]:
clean_train_reviews = []
for review in train1:
    clean_train_reviews.append( review_to_wordlist( review,         remove_stopwords=True ))
    
trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )
print ("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test1:
    clean_test_reviews.append( review_to_wordlist( review,         remove_stopwords=True ))


# In[52]: